In [1]:
# Getting classy
# import the necessary packages
# from tensorflow.keras.layers import LeakyReLU
# from tensorflow.keras.layers import Activation
import tensorflow as tf
from keras.layers import Dense, Reshape, Input, Dropout
from keras.models import Model
from keras import regularizers
# from tensorflow.keras import backend as K
import numpy as np

class flatAutoencoder:
  @staticmethod
  def build(input_shape, layer_dims=(512, 256, 128, 32), 
            enc_dropout_reg = None, dec_dropout_reg = None,
            enc_l2_reg = None, dec_l2_reg = None,
            enc_l1_reg = None, dec_l1_reg = None):
    '''
    enc_dropout_reg: takes tuple size of layer_dims and with the desired dropout
    in the correct index following encoding layer. e.g. (0.5,None,None) if a 
    dropout of 0.5 is desired after the initial layer in given default layer_dims.
    dec_dropout_reg: takes tuple size of layer_dims and desired dropout in 
    index following decoder layers. e.g.(0.5,None,None) in given default
    layer_dims. Currently can't do the last layer but that's small as most models
    performed worse with that layer.

    enc_l2_reg: l2 regularization added to encoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.
    dec_l2_reg: l2 regularization added to decoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.

    enc_l1_reg: l1 regularization added to encoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.
    dec_l1_reg: l1 regularization added to decoder layers if specified. Takes a 
    tuple e.g. (10e-5, None, None) for regularization of the corresponding layer.
    '''
    # define the input to the encoder
    inputs = Input(shape=(input_shape,))
    x = inputs

		# loop over the number of layer_dims to add sets of layers
    for i, dim in enumerate(layer_dims):
      # Check l2 regularization
      if enc_l2_reg is not None:
        if enc_l2_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l2(enc_l2_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)
      
      # Check l1 regularization
      elif enc_l1_reg is not None:
        if enc_l1_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l1(enc_l1_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)

      # Non-regularized layer if no l2/l1
      else:
        x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)

      # Introduce dropout if specified
      if enc_dropout_reg is not None:
        if enc_dropout_reg[i]:
          x = Dropout(enc_dropout_reg[i])(x)
    
    # build encoder model
    encoder = Model(inputs, x, name="encoder")
    
    # Building decoder model
    latentInputs = Input(shape=(layer_dims[-1],))
    x = latentInputs

    # loop over our number of layer_dims again, but in reverse order
    for i, dim in reversed(list(enumerate(layer_dims[:-1]))):
      # Introduce dropout if specified
      if dec_dropout_reg is not None:
        if dec_dropout_reg[i]:
          x = Dropout(dec_dropout_reg[i])(x)

      # Check l2 regularization
      if dec_l2_reg is not None:
        if dec_l2_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l2(dec_l2_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)
      
      # Check l1 regularization
      elif dec_l1_reg is not None:
        if dec_l1_reg[i]:
          x = Dense(dim, activation='relu', 
                    activity_regularizer=regularizers.l1(dec_l1_reg[i]))(x)
        else:
          x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)

      # Non-regularized layer if no l2/l1
      else:
        x = Dense(dim, activation='relu', 
                      activity_regularizer=None)(x)
      # x = LeakyReLU(alpha=0.2)(x) -> might want to try this out later
      
    # apply linear activation (in this case) for output
    outputs = Dense(input_shape, activation='linear')(x)
    
    # build the decoder model
    decoder = Model(latentInputs, outputs, name="decoder")

    # autoencoder is the encoder wrapped by decoder
    autoencoder = Model(inputs, decoder(encoder(inputs)),
                        name="autoencoder")

    # return a 3-tuple of the encoder, decoder, and autoencoder
    return (encoder, decoder, autoencoder)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


TODO:
1. ~Create method to integrate regularization into encoding or decoding layer~
2. ~Fix error that makes layers smaller for some reason (see below layer summaries)~
3. ~Change layer_dims so that it includes the encoding dimension to simplify the code. Ridiculous that I would need to make a special argument just for the encoding dimension - it's the smallest layer. Just use that info.~
4. ~Verify that regularization is built into the layer~
5. Automatic file saving during training.

Future work:
1. Make tests for if more dropouts are provided than can be used
2. Create error if regularization are put in the same spot in the network - e.g. whenever the tuples for both l1 and l2 encoder regularzation are the same
3. Allow different outputs from the model, such as softmax
4. Implement elastic net regularization
5. Create optional argument to share parameters betwee encoding/decoding layers
6. Fix dropout tuple length error

In [1]:
layer_dims=(512, 256, 128, 32)
enc_dropout_reg = (0.5,0.4,0.3,0.2)
dec_dropout_reg = (0.5,0.5,0.5,0.5)
enc_l2_reg = (1e-1,1e-1,1e-1,1e-1)
dec_l2_reg = (1e-2,1e-2,1e-2,1e-2)
# enc_l1_reg = (1e-3,1e-3,1e-3,1e-3)
enc_l1_reg = (None,None,None,None)
# dec_l1_reg = (1e-4,1e-4,1e-4,1e-4)
dec_l1_reg = (None,None,None,None)

# Think I'm going to need to zip forward/backward direction items together
# forward_pass = list(zip(layer_dims[:-1], enc_l1_reg, enc_l2_reg, enc_dropout_reg))
# forward_pass
# backward_pass = list(zip(layer_dims[::-1], dec_l1_reg, dec_l2_reg, dec_dropout_reg))
# backward_pass

[(32, None, 0.01, 0.5),
 (128, None, 0.01, 0.5),
 (256, None, 0.01, 0.5),
 (512, None, 0.01, 0.5)]

In [5]:
forward_pass = list(zip(layer_dims[:-1], enc_l1_reg, enc_l2_reg, enc_dropout_reg))
backward_pass = list(zip(layer_dims[::-1], dec_l1_reg, dec_l2_reg, dec_dropout_reg))

dummy = []
for forward_layer in forward_pass:
  dummy.append('-'.join(str(v) for v in forward_layer))
forward_name = '_'.join(str(v) for v in dummy)

dummy2 = []
for backward_layer in backward_pass:
  dummy2.append('-'.join(str(v) for v in backward_layer))
backward_name = '_'.join(str(v) for v in dummy2)

model_name = '_'.join((forward_name, backward_name))

'512-None-0.1-0.5_256-None-0.1-0.4_128-None-0.1-0.3_32-None-0.01-0.5_128-None-0.01-0.5_256-None-0.01-0.5_512-None-0.01-0.5'

In [8]:
layer_file

'512-256-128-32'

In [3]:
# import tensorflow as tf
# from keras.layers import Input, Dense
# from keras.models import Model
# from keras.layers import Dropout
# from keras import regularizers
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import h5py
import os
import pathlib
from datetime import datetime

date = datetime.today().strftime('%Y-%m-%d')
study_name = "MNIST_Example"
optimizer_metric = "RMS_prop-MSE"

layer_dims=(512, 256, 128, 32)
enc_dropout_reg = (None,None,None,None)
dec_dropout_reg = (None,None,None,None)
enc_l2_reg = (None,None,None,None)
dec_l2_reg = (None,None,None,None)
enc_l1_reg = (None,None,None,None)
dec_l1_reg = (None,None,None,None)

# Filename specifying layers
layer_file = '-'.join(str(v) for v in layer_dims)

# Logic for creating model name
forward_pass = list(zip(layer_dims[:-1], enc_l1_reg, enc_l2_reg, enc_dropout_reg))
backward_pass = list(zip(layer_dims[::-1], dec_l1_reg, dec_l2_reg, dec_dropout_reg))

dummy = []
for forward_layer in forward_pass:
  dummy.append('-'.join(str(v) for v in forward_layer))
forward_name = '_'.join(str(v) for v in dummy)

dummy2 = []
for backward_layer in backward_pass:
  dummy2.append('-'.join(str(v) for v in backward_layer))
backward_name = '_'.join(str(v) for v in dummy2)

model_name = '_'.join((forward_name, backward_name))

# Building the autoencoder
(encoder, decoder, autoencoder) = flatAutoencoder.build(input_shape=784, 
                                      layer_dims=layer_dims,
                                      enc_dropout_reg=enc_dropout_reg, 
                                      dec_dropout_reg=dec_dropout_reg,
                                      enc_l1_reg=enc_l1_reg,
                                      enc_l2_reg=enc_l2_reg,
                                      dec_l1_reg=dec_l1_reg,
                                      dec_l2_reg=dec_l2_reg)

# Configuring optimizer and loss function
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, momentum=0)
autoencoder.compile(optimizer=opt, 
                    loss=tf.keras.losses.MeanSquaredError())

# Path library
file_path = "./drive/My Drive/Lab/BMP AE Models/Models/{} AE/{}/{}/{}".format(layer_file, 
                                                                               study_name, 
                                                                               date, 
                                                                               optimizer_metric)
pathlib.Path(file_path).mkdir(parents=True, exist_ok=True)
drive_file = file_path + '/' + model_name + '_model.h5'

# Early stopping, checkpointing, and csv logger
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)
mc = ModelCheckpoint(filepath=drive_file, 
                     monitor='val_loss', mode='min', 
                     verbose=0, save_best_only=False) # Don't only save best
csv_logger = CSVLogger(drive_file[:-3]+'history_log.csv', append=True)

# Check on models
# print(encoder.summary())
# print(decoder.summary())
# print(autoencoder.summary())

# Loading MNIST data
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Train
history = autoencoder.fit(x_train, x_train,
                          epochs=10,
                          batch_size=512,
                          shuffle=True,
                          validation_data=(x_test, x_test),
                          callbacks=[es, mc, csv_logger])

11493376/11490434 [==============================] - 2s 0us/step
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0614 - val_loss: 0.0453
Epoch 2/10
15872/60000 [======>.......................] - ETA: 0s - loss: 0.0454

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


60000/60000 [==============================] - 1s 11us/step - loss: 0.0416 - val_loss: 0.0369
Epoch 3/10
60000/60000 [==============================] - 1s 12us/step - loss: 0.0348 - val_loss: 0.0350
Epoch 4/10
60000/60000 [==============================] - 1s 12us/step - loss: 0.0313 - val_loss: 0.0299
Epoch 5/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0289 - val_loss: 0.0266
Epoch 6/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0271 - val_loss: 0.0242
Epoch 7/10
60000/60000 [==============================] - 1s 11us/step - loss: 0.0255 - val_loss: 0.0247
Epoch 00007: early stopping


I don't think the callbacks are working.

In [8]:
file_path = "./drive/My Drive/{} AE/{}/{}/{}".format(layer_file, 
                                                                               study_name, 
                                                                               date, 
                                                                               optimizer_metric)
pathlib.Path(file_path).mkdir(parents=True, exist_ok=True)
drive_file = file_path + '/' + model_name + '_model.h5'
print(file_path)
print(drive_file)

FileExistsError: ignored

In [6]:
for layer in encoder.layers:
  # print(layer)
  print(layer.get_config())

{'batch_input_shape': (None, 748), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_5'}
{'name': 'dense_16', 'trainable': True, 'dtype': 'float32', 'units': 512, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dense_17', 'trainable': True, 'dtype': 'float32', 'units': 256, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dense_18', 'trainable': True, 'dtype': 'float32', 'units': 128, 'activation': 'relu', 'use_bias': True

In [18]:
for layer in decoder.layers:
  # print(layer)
  print(layer.get_config())

{'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_6'}
{'name': 'dropout_7', 'trainable': True, 'dtype': 'float32', 'rate': 0.5, 'noise_shape': None, 'seed': None}
{'name': 'dense_18', 'trainable': True, 'dtype': 'float32', 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': {'class_name': 'L1L2', 'config': {'l1': 9.999999747378752e-05, 'l2': 0.0}}, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dropout_8', 'trainable': True, 'dtype': 'float32', 'rate': 0.1, 'noise_shape': None, 'seed': None}
{'name': 'dense_19', 'trainable': True, 'dtype': 'float32', 'units': 256, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': 

I really should look into making a CNN for this. So many less parameters with the CNN.

I now need to:
1. figure out how to choose where and what type of regularization occurs for each layer. The logic of the for loop is going to get a little more complicated by having 

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dropout

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=None)(input_img)
encoded = Dropout(0.4)(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.summary()

# autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# autoencoder.fit(x_train, x_train,
#                 epochs=50,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 784)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 32)                25120     
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________
